In [1]:
# Importing the required libraries 
import nltk
import numpy as np
import itertools

# Downloading and importing the brown corpus
nltk.download('brown')
from nltk.corpus import brown

#Getting the tagged sentences
sent_tag = brown.tagged_sents()
brown_sent_tag=[]
for s in sent_tag:
    s.insert(0,('##','##'))
    s.append(('&&','&&'))
    brown_sent_tag.append(s)

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [38]:
#Splitting the data for train and test
split_num = int(len(brown_sent_tag)*0.9)
train_data = brown_sent_tag[0:split_num]
test_data = brown_sent_tag[split_num:]

In [39]:
# MAking a dict() variable to store TAG , word and its count in the following fashion:
# {'TAG' : {word : 'count of word with TAG'}  }

train_word_tag = {}
for data in train_data:
    #print(data)
    for word, tag in data:
        #print(word,tag)
        #print('------')
        word = word.lower()
        if tag in train_word_tag:
            if word in train_word_tag[tag]:
                train_word_tag[tag][word] +=1
            else:
                train_word_tag[tag][word]=1
        else:
            train_word_tag[tag] ={word:1}

In [40]:
# Making the emission probability matrix

# Making a nested disctionary to in the following fashion:
# { Tag : {word: emission probability form that tag' s state} }

# This is the variable to store the dictionaty
train_emission_prob_matrix = {}
for tag in train_word_tag.keys():
    
    # creating the blank nested dictionary for {word : emission prob}
    train_emission_prob_matrix[tag] = {}
    
    # calculating number of word with that tag
    count = sum(train_word_tag[tag].values())
    
    for word in train_word_tag[tag].keys():
        
        # Calculating the emmission prob of each word for that state or Tag
        train_emission_prob_matrix[tag][word] = train_word_tag[tag][word]/count
#train_emission_prob_matrix

In [41]:
# calculating sequence of tags as bigram in the following fashion
# {TAG of 1st word of a given bigram i.e. (T1) :{Tag of 2nd word of a given bigram i.e (T2) : count of (T1 followed by T2) }}

bigram_of_tag = {}

for sentence in train_data:
    
    # sentence consist of a Tagged-sentence so to get the bigram of that
    generated_bigrams = list(nltk.bigrams(sentence))
    
    for bigram1 ,bigram2 in generated_bigrams:
        
        ## checking if tag of first word of bigram exist 
            ## if yes then check if the tag of 2nd word of the bigram exixst
                    ## If es then increament its value by 1
            ## else assign the value to the new tag as 1
        ## else create the new entry with value 1 
        if bigram1[1] in bigram_of_tag:
            if bigram2[1] in bigram_of_tag[bigram1[1]]:
                bigram_of_tag[bigram1[1]][bigram2[1]] += 1
            else:
                bigram_of_tag[bigram1[1]][bigram2[1]] = 1
        else:
            bigram_of_tag[bigram1[1]] = { bigram2[1] : 1}
#print(bigram_of_tag)

In [42]:
prob_of_bigram_tag = {}

for tag in bigram_of_tag.keys():
    
    # creating a nested dictioonary for each tag
    prob_of_bigram_tag[tag] = {}
    
    # summing up all the occurance of the tag
    count = sum(bigram_of_tag[tag].values())
    
    for tag2 in bigram_of_tag[tag].keys():
        prob_of_bigram_tag[tag][tag2] = bigram_of_tag[tag][tag2]/count
#prob_of_bigram_tag 

In [43]:
# Now I will find all possible tag which was given to a word in both test and train data

possible_tags_of_word = {}

for data in train_data:
    for word, tag in data:
        
        # lowering the the capital letters
        word = word.lower()
        
        # checking if the word in already in the dictionary
            ## if yes the check if that is already in the list
                ## if yes then continue
                ## if not then append it to the list
            ## if not the create a new nested dict with {word:[tag]}  
            
        if word in possible_tags_of_word:
            if tag not in possible_tags_of_word[word]:
                possible_tags_of_word[word].append(tag)                
        else:
            temp=[]
            temp.append(tag)
            possible_tags_of_word[word] = temp
#print(possible_tags_of_word)

# Now doing the same thing for the test data
for data in test_data:
    for word, tag in data:
        
        # lowering the the capital letters
        word = word.lower()
        
        # checking if the word in already in the dictionary
            ## if yes the check if that is already in the list
                ## if yes then continue
                ## if not then append it to the list
            ## if not the create a new nested dict with {word:[tag]}  
            
        if word in possible_tags_of_word:
            if tag not in possible_tags_of_word[word]:
                possible_tags_of_word[word].append(tag)                
        else:
            temp=[]
            temp.append(tag)
            possible_tags_of_word[word] = temp
#print(possible_tags_of_word)

In [44]:
# pre processing the test data
# Removing the tags from the test data set

# The following structure is used
## test_word=[[word1, word2,word3...],
##                     [word1, word2, word3....]]   where each row is one sentence.

## in similar fashion the tags are stored
## test_tags = [[tag1, tag2, tag3 ,.......],
##                      [tag1, tag2, tag3,.......]] where each row i s one sentence 

test_words = []
test_tags = []

for data in test_data:
    temp_words = []
    temp_tags =[]
    for word, tag in data:
        temp_words.append(word.lower())
        temp_tags.append(tag)
    test_words.append(temp_words)
    test_tags.append(temp_tags)
#test_tags

In [45]:
#Executing the Viterbi Algorithm
predicted_tags = []                #intializing the predicted tags
for x in range(len(test_words)):   # for each tokenized sentence in the test data
    s = test_words[x]
    #storing_values is a dictionary which stores the required values
    #ex: storing_values = {step_no.:{state1:[previous_best_state,value_of_the_state]}}                
    storing_values = {}              
    for q in range(len(s)):
        step = s[q]
        #for the starting word of the sentence
        if q == 1:                
            storing_values[q] = {}
            tags = possible_tags_of_word[step]
            for t in tags:
            #this is applied since we do not know whether the word in the test data is present in train data or not
                try:
                    storing_values[q][t] = ['##',bigram_tag_prob['##'][t]*train_emission_prob[t][step]]
                    #if word is not present in the train data but present in test data we assign a very low probability of 0.0001
                except:
                    storing_values[q][t] = ['##',0.0001]#*train_emission_prob[t][step]]
    
         #if the word is not at the start of the sentence
        if q>1:
            storing_values[q] = {}
            previous_states = list(storing_values[q-1].keys())   # loading the previous states
            current_states  = possible_tags_of_word[step]               # loading the current states
            #calculation of the best previous state for each current state and then storing
            #it in storing_values
            for t in current_states:                             
                temp = []
                for pt in previous_states:                         
                    try:
                        temp.append(storing_values[q-1][pt][1]*bigram_tag_prob[pt][t]*train_emission_prob[t][step])
                    except:
                        temp.append(storing_values[q-1][pt][1]*0.0001)
        
                max_temp_index = temp.index(max(temp))
                best_pt = previous_states[max_temp_index]
                storing_values[q][t]=[best_pt,max(temp)]
    #print(storing_values)
    #print('---------------')
  #Backtracing to extract the best possible tags for the sentence
    pred_tags = []
    total_steps_num = storing_values.keys()
    #print(total_steps_num)
    last_step_num = max(total_steps_num)
    for bs in range(len(total_steps_num)):
        step_num = last_step_num - bs
        if step_num == last_step_num:
            pred_tags.append('&&')
            pred_tags.append(storing_values[step_num]['&&'][0])
        if step_num<last_step_num and step_num>0:
            pred_tags.append(storing_values[step_num][pred_tags[len(pred_tags)-1]][0])
    predicted_tags.append(list(reversed(pred_tags)))

storing_values

{1: {'IN': ['##', 0.0001], 'IN-HL': ['##', 0.0001], 'IN-TL': ['##', 0.0001]},
 2: {'WDT': ['IN', 1e-08],
  'WDT-HL': ['IN', 1e-08],
  'WPS-TL': ['IN', 1e-08],
  'WPO-TL': ['IN', 1e-08],
  'WDT-NC': ['IN', 1e-08],
  'WPS': ['IN', 1e-08]},
 3: {'PPSS': ['WDT', 1e-12],
  'NN': ['WDT', 1e-12],
  'PPSS-NC': ['WDT', 1e-12],
  'NN-TL': ['WDT', 1e-12],
  'NP': ['WDT', 1e-12],
  'NIL': ['WDT', 1e-12],
  'PPSS-HL': ['WDT', 1e-12]},
 4: {'BEDZ': ['PPSS', 1e-16],
  'BEDZ-HL': ['PPSS', 1e-16],
  'BEDZ-NC': ['PPSS', 1e-16]},
 5: {'JJ': ['BEDZ', 1.0000000000000001e-20]},
 6: {'TO': ['JJ', 1.0000000000000001e-24],
  'IN': ['JJ', 1.0000000000000001e-24],
  'IN-HL': ['JJ', 1.0000000000000001e-24],
  'TO-HL': ['JJ', 1.0000000000000001e-24],
  'IN-TL': ['JJ', 1.0000000000000001e-24],
  'TO-TL': ['JJ', 1.0000000000000001e-24],
  'NPS': ['JJ', 1.0000000000000001e-24],
  'NIL': ['JJ', 1.0000000000000001e-24],
  'QL': ['JJ', 1.0000000000000001e-24],
  'TO-NC': ['JJ', 1.0000000000000001e-24],
  'IN-NC': ['JJ',

In [72]:
#Calculating the accuracy based on tagging each word in the test data.
right = 0 
wrong = 0

check_dict = {}
check_word_dict = {}
check_sent_dict ={}
for i in range(len(test_tags)):
    gt = test_tags[i]
    pred = predicted_tags[i]
    for h in range(len(gt)):
        if gt[h] == pred[h]:
            right = right+1
        else:
            if gt[h] +" " +pred[h] in check_dict:
                check_dict[gt[h] +" " +pred[h]] +=1
                check_sent_dict[gt[h] +" " +pred[h]].append( [" ".join(test_words[i])])
                if test_words[i][h]  not in check_word_dict[gt[h] +" " +pred[h]]:
                    check_word_dict[gt[h] +" " +pred[h]].append(test_words[i][h])
            else:
                check_dict[gt[h] +" " +pred[h]] =1
                check_word_dict[gt[h] +" " +pred[h]] = [test_words[i][h]]
                check_sent_dict[gt[h] +" " +pred[h]] = [" ".join(test_words[i])]
            #print("Sentence")
            #print(test_words[h], pred[h])
            #print('Predicted tags',pred[h])
            #print('Ground_truth', gt[h])
            wrong = wrong +1 
print(check_word_dict)
print('Accuracy on the test data is: ',right/(right+wrong))
print('Loss on the test data is: ',wrong/(right+wrong))

{'RB IN': ['about', 'besides', 'around', 'inside', 'outside', 'since', 'below', 'along', 'rather', 'by', 'behind', 'above', 'across', 'against', 'alongside', 'with', 'beneath'], 'NN JJ': ['dear', 'fugitive', 'vernacular', 'sound', 'concrete', 'brief', 'cold', 'crisp', 'dead', 'fit', 'grave', 'ancient', 'blonde', 'flush', 'chill', 'public', 'miniature', 'mosaic', 'contemporary', 'giant', 'classic', 'equal', 'intellectual', 'upright', 'evil', 'hollow', 'present', 'lavender', 'magic', 'limp', 'token', 'good', 'special', 'secret', 'vaginal', 'dark', 'tart', 'melancholy', 'common', 'pink', 'tan', 'average', 'steep', 'bronze', 'potential', 'blind', 'private', 'contrary', 'tense', 'routine', 'alternative', 'maximum', 'convertible', 'quiet', 'intimate', 'rich', 'reactionary', 'associate', 'blank', 'boon', 'anatomical', 'social', 'indefinite', 'attendant', 'equivalent', 'veteran', 'abstract', 'unconscious', 'model', 'collective', 'safe'], 'VBD VBN': ['presented', 'valued', 'enjoyed', 'circulate

In [79]:
# ploting the errros
if 'NN PP$$' in check_dict:
    check_dict.pop('NN PP$$')
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(80, 20), dpi=320, facecolor='w', edgecolor='k')
plt.xticks(rotation=90)
plt.xlabel('TAGS -->')
plt.ylabel('Frequency -->')
plt.bar(list(check_dict.keys()), check_dict.values(),color='g')

<BarContainer object of 332 artists>

10 tag that are mostly mis-tagged

In [55]:
temp = sorted(check_dict.items(), key=lambda item: item[1], reverse = True)


In [78]:
from prettytable import PrettyTable
t = PrettyTable(['Ground-Truth', 'Predicted-Tag','Misclassified-Word' ,'sent','frequency'])
for i in range(0,10):
    x,y = temp[i][0].split()
    t.add_row([x, y,check_word_dict[temp[i][0]],check_sent_dict[temp[i][0]],temp[i][1]])
    
#print(t)   

In [76]:

# writing to CSV 
import csv 
# name of csv file  
filename = "result1.csv"
    
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    csvwriter.writerow(['Ground-Truth', 'Predicted-Tag','Misclassified-Word' ,'Sentence','frequency'])  
    for i in range(len(temp)):
        x,y = temp[i][0].split()
        rows = [x, y,check_word_dict[temp[i][0]],check_sent_dict[temp[i][0]],temp[i][1]]
    
  
        # writing the data rows  
        csvwriter.writerow(rows) 

In [77]:
import pandas as pd
df = pd.read_csv('result1.csv')
df.head(10)

,Ground-Truth,Predicted-Tag,Misclassified-Word,Sentence,frequency
0,VBD,VBN,"['presented', 'valued', 'enjoyed', 'circulated...","['## delphine presented her cheek for a kiss ,...",941
1,IN,TO,['to'],['## the physician led the horses to the stabl...,796
2,VBN,VBD,"['prepared', 'led', 'pinched', 'tumbled', 'pul...",['## i want the room in the attic prepared for...,591
3,NN,NN-TL,"['house', 'youth', 'mansion', 'jury', 'city', ...","['## `` dandy is to be our house guest , louis...",584
4,NN,VB,"['smell', 'mass', 'face', 'hate', 'hope', 'eas...","[""## but he liked the smell of delphine's perf...",547
5,VB,NN,"['doubt', 'glance', 'return', 'stay', 'move', ...","[""## the lalauries were at the top rung of the...",455
6,PPO,PPS,['it'],"['## delphine presented her cheek for a kiss ,...",387
7,PPO,PP$,['her'],['## a stringed orchestra played softly behind...,334
8,DT,CS,['that'],['## it was just as well that the ignorant dan...,325
9,NN-TL,NN,"['street', 'senator', 'village', 'quartet', 'm...","[""## when he finally left the sinister mansion...",270
